In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 41 kB 185 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30535 sha256=d2bd88d83580b0646d33834ec380c51e5ce400dc1ac14293a755cc5e2d42d4b8
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=012311514f2b21e1280faf04318e9b36932c60f1b8cbc38c23db6b76b302ee70
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=81f9c90fd001cd97c95244c9f20386247ff315d894c48466ab60b6a96f3d79b4
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow py-params
     |████████████████████████████████| 1.2 MB 5.0 MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 46.7 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
     |████████████████████████████████| 596 kB 64.1 MB/s 
     |████████████████████████████████| 895 kB 63.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Model,Sequential, Input, load_model
from keras.layers import *
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
#from transformers import TFBertModel, BertTokenizer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras import metrics

from tensorflow.keras.applications.inception_v3 import InceptionV3
from itertools import combinations as comb
import ntpath
import glob
import math
from sklearn import metrics
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import OneHotEncoder 
#from transformers import TFBertModel, DistilBertTokenizerFast


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/My Drive/"

/content/drive/My Drive


In [ ]:
df=pd.read_csv('./Tweets.csv')

In [ ]:
len(df)

14640

In [ ]:
positive=df[df['airline_sentiment']=='positive'].text
neutral=df[df['airline_sentiment']=='neutral'].text
negative=df[df['airline_sentiment']=='negative'].text

In [ ]:
# convert Sentiments to 0,1,2
def convert_Sentiment(sentiment):
    if  sentiment == "positive":
        return 2
    elif sentiment == "neutral":
        return 1
    elif sentiment == "negative":
        return 0

In [ ]:
# Apply convert_Sentiment function
df.airline_sentiment = df.airline_sentiment.apply(lambda x : convert_Sentiment(x))

In [ ]:
 import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Remove stop words
def remove_stopwords(text):
    text = ' '.join([word for word in text.split() if word not in (stopwords.words('english'))])
    return text

# Remove url  
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

# Remove punct
def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

# Remove html 
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

# Remove @username
def remove_username(text):
    return re.sub('@[^\s]+','',text)

# Remove emojis
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


# Decontraction text
def decontraction(text):
    text = re.sub(r"won\'t", " will not", text)
    text = re.sub(r"won\'t've", " will not have", text)
    text = re.sub(r"can\'t", " can not", text)
    text = re.sub(r"don\'t", " do not", text)
    
    text = re.sub(r"can\'t've", " can not have", text)
    text = re.sub(r"ma\'am", " madam", text)
    text = re.sub(r"let\'s", " let us", text)
    text = re.sub(r"ain\'t", " am not", text)
    text = re.sub(r"shan\'t", " shall not", text)
    text = re.sub(r"sha\n't", " shall not", text)
    text = re.sub(r"o\'clock", " of the clock", text)
    text = re.sub(r"y\'all", " you all", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"n\'t've", " not have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'d've", " would have", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ll've", " will have", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'re", " are", text)
    return text  

# Seperate alphanumeric
def seperate_alphanumeric(text):
    words = text
    words = re.findall(r"[^\W\d_]+|\d+", words)
    return " ".join(words)

def cont_rep_char(text):
    tchr = text.group(0) 
    
    if len(tchr) > 1:
        return tchr[0:2] 

def unique_char(rep, text):
    substitute = re.sub(r'(\w)\1+', rep, text)
    return substitute

def char(text):
    substitute = re.sub(r'[^a-zA-Z]',' ',text)
    return substitute

# combaine negative reason with  tweet (if exsist)
df['final_text'] = df['negativereason'].fillna('') + ' ' + df['text'] 


# Apply functions on tweets
df['final_text'] = df['final_text'].apply(lambda x : remove_username(x))
df['final_text'] = df['final_text'].apply(lambda x : remove_url(x))
df['final_text'] = df['final_text'].apply(lambda x : remove_emoji(x))
df['final_text'] = df['final_text'].apply(lambda x : decontraction(x))
df['final_text'] = df['final_text'].apply(lambda x : seperate_alphanumeric(x))
df['final_text'] = df['final_text'].apply(lambda x : unique_char(cont_rep_char,x))
df['final_text'] = df['final_text'].apply(lambda x : char(x))
df['final_text'] = df['final_text'].apply(lambda x : x.lower())
df['final_text'] = df['final_text'].apply(lambda x : remove_stopwords(x))

In [ ]:
X = df['final_text']
y = df['airline_sentiment']

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import bert
import transformers
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

In [ ]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1", trainable=True)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1'.
INFO:absl:Downloaded https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1, Total size: 1.26GB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1'.


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

def encoder(sentences):
  ids = []
  for sentence in sentences:
    encoding = tokenizer.encode_plus(
    sentence,
    max_length=16,
    truncation = True,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=False)
    ids.append(encoding['input_ids'])
  return ids

#Train test split
train_sents,test_sents, train_labels, test_labels  = train_test_split(X,y,test_size=0.15)

train_ids = encoder(train_sents)
test_ids = encoder(test_sents) 

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
train_ids = tf.convert_to_tensor(train_ids)
test_ids = tf.convert_to_tensor(test_ids)
test_labels = tf.convert_to_tensor(test_labels)
train_labels = tf.convert_to_tensor(train_labels)

In [ ]:
max_len=50
bert_encoder = TFBertModel.from_pretrained('bert-base-uncased')
input_word_ids = tf.keras.Input(shape=(16,), dtype=tf.int32, name="input_word_ids")  
embedding = bert_encoder([input_word_ids])
dense = tf.keras.layers.Lambda(lambda seq: seq[:, 0, :])(embedding[0])
dense = tf.keras.layers.Dense(128, activation='relu',kernel_regularizer='l2')(dense)
dense = tf.keras.layers.Dropout(0.2)(dense)   
output = tf.keras.layers.Dense(1, activation='sigmoid',kernel_regularizer='l2')(dense)    

model = tf.keras.Model(inputs=[input_word_ids], outputs=output)  

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
#sBERT = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
# sBERT.compile(SGD(lr=learning_rate, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])
model.compile(Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
train_history = model.fit(x = train_ids, y = train_labels, epochs = 3, verbose = 1, batch_size = 32, validation_data = (test_ids, test_labels))
model.summary()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/3


389/389 [==============================] - 72s 147ms/step - loss: -7.5087 - accuracy: 0.7862 - val_loss: -22.0358 - val_accuracy: 0.8174
Epoch 2/3
389/389 [==============================] - 56s 145ms/step - loss: -3.3612 - accuracy: 0.4144 - val_loss: 2.5716 - val_accuracy: 0.1926
Epoch 3/3
389/389 [==============================] - 56s 144ms/step - loss: 2.3321 - accuracy: 0.2693 - val_loss: 2.1153 - val_accuracy: 0.1926
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 16)]             0         
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  109482240
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             16, 768),                           
                             

In [ ]:
test_pred = sBERT.predict(val_input)


In [ ]:
print(" accuracy_score: {}".format(accuracy_score(val_labels,test_pred.round())))

 accuracy_score: 0.19808743169398907


In [ ]:
bert_encoder = Model(input_word_ids, model.get_layer('lambda').output)
bert_encoder.summary()


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 16)]             0         
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  109482240
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             16, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                           

In [ ]:
train_embed = bert_encoder.predict(train_ids)
test_embed = bert_encoder.predict(test_ids)

In [ ]:
np.array(train_embed).shape

(12444, 768)

In [ ]:
def generate_imp_space(X_train, Y_train, X_test, imp_feature_size, foz):
  X_train=pd.DataFrame(X_train)
  X_test=pd.DataFrame(X_test)
  imp_train_data = X_train.values
  imp_test_data = X_test.values
  Y_train=pd.DataFrame(Y_train)
  d = len(X_train.columns)
  num_class=len(np.unique(Y_train))
  # print("____1_____")
  for i in range(0,imp_feature_size*foz):
    Xindis = np.random.permutation(d)
    for j in range(0,d-(foz-1),foz):  #d/foz kadar doner
      sX = np.random.permutation(num_class)
      s1 = sX[0]
      # print("____2_____")
      s1data = X_train[X_train.index.isin(Y_train[Y_train == str(s1)].index)]
      s2data = X_train[~X_train.index.isin(Y_train[Y_train == str(s1)].index)]
      s1data = s1data.iloc[:,Xindis[j:j+(foz)]]
      s2data = s2data.iloc[:,Xindis[j:j+(foz)]] # s1 vs all other classes, #foz feature
      # print("____3_____")
      s1label = np.ones((s1data.values.shape[0],1),dtype=int)
      s2label = -1*np.ones((s2data.values.shape[0],1),dtype=int)
      Wdata = np.concatenate((s1data,s2data))
      # print("____4_____")
      
      Wdata = x2fx(Wdata)
      Wlabel = np.concatenate((s1label,s2label))
      W = np.matmul(np.matmul(np.linalg.pinv(np.matmul(Wdata.T, Wdata)),Wdata.T),Wlabel)
      
      WW = x2fx(X_train.iloc[:,Xindis[j:j+(foz)]].values)
      imp_train_data = np.concatenate((imp_train_data, np.matmul(WW,W)),axis=1)
      
      TT = x2fx(X_test.iloc[:,Xindis[j:j+(foz)]].values)
      imp_test_data = np.concatenate((imp_test_data, np.matmul(TT,W)),axis=1)
    
  return imp_train_data,imp_test_data
  

In [ ]:
def x2fx(x, model='quadratic'):
    linear = np.c_[np.ones(x.shape[0]), x]
    if model == 'linear':
        return linear
    if model == 'purequadratic':
        return np.c_[linear, x**2]
    interaction = np.array([x[:,i]*x[:,j] for i, j in comb(range(x.shape[1]), 2)]).T
    if model == 'interaction':
        return np.c_[linear, interaction]
    if model == 'quadratic':
        return np.c_[linear, interaction, x**2]
def MajorityVoting(votes):
  results = []
  for i in range(0,votes.shape[1]):
    values, counts = np.unique(votes[:,i], return_counts=True)
    results.append(values[np.argmax(counts)])
  return np.array(results)

In [ ]:
foz=4
imp_feature_size=1
acc_imp=[]

acc_norm=[]
acc_incept=[]
imp_tree_predicts=[]
tree_predicts=[]
train_embed=pd.DataFrame(train_embed)
d=len(train_embed.columns)
for i in range(0,5):
    imp_tr, imp_ts = generate_imp_space(train_embed, train_labels, test_embed, imp_feature_size, foz)
    imp_d = imp_tr.shape[1]
    imp_sel_d = 2* round(math.log2(imp_d)) #feature
    sel_d = 2*round(math.log2(d))
      
    imp_tree = XGBClassifier(max_depth=imp_sel_d, n_estimators=50,random_state=42)
   
    imp_tree.fit(imp_tr, train_labels)
    imp_tree_predicts.append(imp_tree.predict(imp_ts))


    tree = XGBClassifier(max_depth=sel_d, n_estimators=50,random_state=42)
    
    tree.fit(train_embed, train_labels)
    tree_predicts.append(tree.predict(test_embed))
result_imp=MajorityVoting(np.array(imp_tree_predicts))
result_norm=MajorityVoting(np.array(tree_predicts))
acc_imp.append(metrics.accuracy_score(test_labels, result_imp))
acc_norm.append(metrics.accuracy_score(test_labels, result_norm))
#result_norm=encoder.fit_transform(result_norm)
#result_imp=encoder.fit_transform(result_imp)

ValueError: ignored

In [ ]:
train_embed=pd.DataFrame(train_embed).values
test_embed=pd.DataFrame(test_embed).values
tree = XGBClassifier(max_depth=sel_d, n_estimators=50,random_state=42)
tree.fit(train_embed,train_labels)

XGBClassifier(max_depth=20, n_estimators=50, objective='multi:softprob',
              random_state=42)

In [ ]:
r=tree.predict(test_embed)
metrics.accuracy_score(test_labels, r)

0.8474499089253188

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfclassifier = RandomForestClassifier(n_estimators = 20, criterion = 'entropy', random_state = 0)
rfclassifier.fit(train_embed, train_labels)
test_predrf = rfclassifier.predict(test_embed).round().astype(int)
print("RF: ", accuracy_score(test_labels, test_predrf))

RF:  0.8146630236794171


In [ ]:
print(acc_imp)
print(acc_norm)

[0.8169398907103825]
[0.8205828779599271]


In [ ]:
print("GB")
print(acc_imp)
print(acc_norm)

GB
[0.7190346083788707]
[0.726775956284153]


In [ ]:
result_imp=MajorityVoting(np.array(imp_tree_predicts))
metrics.accuracy_score(test_labels, result_imp)

0.8397085610200364